<a href="https://colab.research.google.com/github/spaceBearAmadeus/OUTDATED-studentNeedsNavigator/blob/main/rose_composer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!sudo apt install -y fluidsynth

In [ ]:
!pip install --upgrade pyfluidsynth

In [ ]:
!pip install pretty_midi

In [ ]:
import collections
import datetime
import fluidsynth
import glob
import numpy as np
import pathlib
import pandas as pd
import pretty_midi
import seaborn as sns
import tensorflow as tf

from IPython import display
from matplotlib import pyplot as plt
from typing import Dict, List, Optional, Sequence, Tuple

In [ ]:
tf.random.set_seed(42)
np.random.seed(42)

AUDIO_DIR = pathlib.Path("data/maestro-v2.0.0")
ANNOTATIONS_FILE = "/content/data/maestro-v2.0.0.csv"

if not AUDIO_DIR.exists():
  tf.keras.utils.get_file(
      'maestro-v2.0.0-midi.zip',
      origin='https://storage.googleapis.com/magentadata/datasets/maestro/v2.0.0/maestro-v2.0.0-midi.zip',
      extract=True,
      cache_dir='.', cache_subdir='data',
  )

In [ ]:
class MaestroDataset:
  def __init__(self, annotations_file, audio_dir):
    self.annotations = pd.read_csv(f"{annotations_file}")
    self.audio_dir = audio_dir 
    self._SAMPLING_RATE = 16000

  def __len__(self):
    return len(self.annotations)

  def __getitem__(self, index):
    audio_sample_path = self._get_audio_sample_path(index)
    label = self._get_audio_sample_label(index)
    return audio_sample_path, label 

  def _get_audio_sample_path(self, index):
    filenames = glob.glob(str(self.audio_dir/'**/*.mid*'))
    return filenames[index]

  def _get_audio_sample_label(self, index):
    return self.annotations.iloc[index, 6]
  
  '''retrieves pretty_midi object'''
  def get_pretty_midi(self, file_index:int):
    pm = pretty_midi.PrettyMIDI(self.__getitem__(file_index)[0])
    return pm

  def display_audio(self, pm:pretty_midi.PrettyMIDI, seconds=30):
    waveform = pm.fluidsynth(fs=self._SAMPLING_RATE)
    # Take a sample of the generated waveform to mitigate kernel resets
    waveform_short = waveform[:seconds*self._SAMPLING_RATE]
    return display.Audio(waveform_short, rate=self._SAMPLING_RATE)
      


rose = MaestroDataset(ANNOTATIONS_FILE, AUDIO_DIR)

In [ ]:
pm = rose.get_pretty_midi(11)

In [ ]:
rose.display_audio(pm)